# Data augmentation 

Este é um dos notebooks contidos na pasta [DATA AUGMENTATION](../DATA%20AUGMENTATION/). A fim de contemplarmos todos os testes e alterações, implementamos mais dois notebooks que possuem, além do Data Augmentation, os Pré-Processamentos de imagens criados nas duas sprints anteriores.

Portanto, este é o notebook que tem e documenta as diversas manipulações de imagens, a partir das imagens encontradas em [tci_tifs](../../../../data/SPRINT%201/dataset_inteli/tci_tifs/) e [masks](../../../../data/SPRINT%201/dataset_inteli/masks/). Você pode encontrar os notebooks que contemplam o processo de Data Augmentation aplicado ao pré processamento de imagens nos outros colabs abaixo:

- [Implementação de Data Augmentation no processamento de imagens tif]().
- [Implementação de Data Augmentation no processamento de imagens usando KMeans e Sharpen]()

O artigo "Automatic Rural Road Centerline Detection and Extraction from Aerial Images for a Forest Fire Decision Support System" utilizou técnicas de data augmentation para aumentar o conjunto de dados inicial, que consistia em 486 imagens ortorretificadas. Para ampliar o tamanho do dataset e melhorar o treinamento do modelo, os seguintes métodos de data augmentation foram aplicados:

1. **Rotação:** As imagens foram rotacionadas em noventa graus quatro vezes.
2. **Espelhamento:** A operação de espelhamento foi realizada.

Além disso, implementamos o método de recorte (crop), onde a partir de imagens de 600x600, fazemos recortes de quadrantes de 200x200, inspirado no artigo "Image Preprocessing for Computer Vision Tasks in Python Using OpenCV and Tensorflow".

Essas táticas foram cruciais para aumentar significativamente o tamanho do conjunto de dados e aprimorar o processo de treinamento do modelo, como descrito nos estudos.

_Referências:_
AUTOMATIC Rural Road Centerline Detection and Extraction from Aerial Images for a Forest Fire Decision Support System. Remotesensing, 2024. Disponível em: <https://www.mdpi.com/2072-4292/15/1/271>. Acesso em: 24 maio 2024.

BTD. Image Preprocessing for Computer Vision Tasks in Python Using OpenCV and Tensorflow. Medium, 2023. Disponível em: <https://medium.com/plans?source=upgrade_membership---post_counter--be5ddbeb6c09>. Acesso em: 24 maio 2024.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
import os
from PIL import Image

# Mount Google Drive
drive.mount('/content/drive')

In [ ]:
# Função auxiliar para criação de diretórios através de um caminho
def create_dirs(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
# Carregar máscaras correspondentes para as imagens mescladas e redimensionar para 600x600
def load_masks(masks_dir, target_size=(600, 600)):
    masks = []
    ordered_masks = sorted(os.listdir(masks_dir))
    mask_filenames = []

    for mask_name in ordered_masks:
        mask_path = os.path.join(masks_dir, mask_name)
        mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
    
        if mask is not None:

            if mask.shape[:2] != target_size:
                mask = cv2.resize(mask, target_size[::-1])
            
            masks.append(mask)
            mask_filenames.append(os.path.splitext(mask_name)[0])
    
        else:
            print(f"Falha ao carregar máscara: {mask_path}")
    
    return masks, mask_filenames

# Função para carregar imagens
def load_images(image_dir, target_size=(600, 600)):
    images = []
    ordered_images = sorted(os.listdir(image_dir))
    image_filenames = []
    for image_name in ordered_images:
        image_path = os.path.join(image_dir, image_name)
        image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        if image is not None:
            if image.shape[:2] != target_size:
                image = cv2.resize(image, target_size[::-1])
            images.append(image)
            image_filenames.append(os.path.splitext(image_name)[0])
        else:
            print(f"Falha ao carregar imagem: {image_path}")
    return images, image_filenames

# Caminhos
masks_dir = '/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli/masks'
image_dir = '/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli/tci_tifs'

# Carregar imagens e máscaras
masks, mask_filenames = load_masks(masks_dir)
images, image_filenames = load_images(image_dir)


In [ ]:

# Garantir que as máscaras e as imagens estejam na mesma ordem
assert mask_filenames == image_filenames, "A ordem das máscaras e imagens não corresponde!"

class ImageProcessingPipeline:
	def __init__(self, images, masks):
		self.images = images
		self.masks = masks

	def crop_image(self, image, crop_size=(200, 200)):
		crops = []

		for i in range(0, image.shape[0], crop_size[0]):
			for j in range(0, image.shape[1], crop_size[1]):
				crop = image[i:i+crop_size[0], j:j+crop_size[1]]
				if crop.shape[0] == crop_size[0] and crop.shape[1] == crop_size[1]:
					crops.append(crop)

		return crops

	def augment_images(self, image):
		aug_images = []
		for angle in [0, 90, 180, 270]:
			rotated = self.rotate_image(image, angle)
			aug_images.append(rotated)
			aug_images.append(cv2.flip(rotated, 1))

		return aug_images

	@staticmethod
	def rotate_image(image, angle):
		(h, w) = image.shape[:2]
		center = (w // 2, h // 2)
		M = cv2.getRotationMatrix2D(center, angle, 1.0)
		return cv2.warpAffine(image, M, (w, h))

	def process_and_save_images_and_masks(self, output_dir):
		create_dirs(output_dir)
		count = 0
		
		for img, mask in zip(self.images, self.masks):
			cropped_images = self.crop_image(img)
			cropped_masks = self.crop_image(mask)

			for crop_img, crop_mask in zip(cropped_images, cropped_masks):
				augmented_imgs = self.augment_images(crop_img)
				augmented_masks = self.augment_images(crop_mask)
		
				for aug_img, aug_mask in zip(augmented_imgs, augmented_masks):
					aug_img = Image.fromarray((aug_img * 255).astype(np.uint8))  # Converter de volta para uint8
					aug_mask = Image.fromarray(aug_mask)
					
					aug_img.save(os.path.join(output_dir, f'processed_image_{count}.tif'))
					aug_mask.save(os.path.join(output_dir, f'processed_mask_{count}.png'))
		
					count += 1

	def show_image(self, image):
		plt.imshow(image, cmap='gray')
		plt.axis('off')
		plt.show()

# Processar e aumentar imagens e máscaras
output_dir = '/content/drive/Shared drives/Grupo T de Tech/Data/datasets_sprint3/dataset_inteli_processed'
pipeline = ImageProcessingPipeline(images, masks)
pipeline.process_and_save_images_and_masks(output_dir)

# Mostrar algumas das imagens processadas
for img in images[:8]:  # Mostra as primeiras 8 imagens processadas
	pipeline.show_image(img)

Caso haja necessidade de executar o código, devido ao peso dos datasets optamos por mante-los no google drive, contudo podem ser acessados através dos links abaixo:

- [Conjunto de imagens geradas através de data augmentation](https://drive.google.com/drive/folders/1nblgJuT7ZmEZmh6qrF_cbdllCeZQswHM?usp=sharing)